In [1]:
import re
import datetime
import numpy as np
import pandas as pd 
import os
import sys
import itertools
import io

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.utils import *

### Establishes General Path

In [3]:
local_path = "/your/path/here"

In [4]:
path = '/Data/chicago-police-data/import'
db = dropbox_handler()
db.list_files(path+'/input')

['complaints-merged-2015',
 'complaints-cpd-2016-jun',
 'complaints-cpd-2016-oct',
 'complaints-cpd-2016-nov',
 'complaints-ipra-2016-apr',
 'shootings-ipra-may2016',
 'shootings-cpd-feb2016',
 'complaints-cpd-2016-dec',
 'TRRs']

In [5]:
in_path = path + '/input/complaints-cpd-2016-oct/'
out_path =local_path + '/output/complaints-cpd-2016-oct/'

In [6]:
location_code = db.download_file(path+'/doc/','Location_Code_Dictionary.csv')
location_code.dropna(how='all', inplace=True)
location_code['Location_Code']=location_code['Location_Code'].astype(int).astype(str)

def padding(value):
    if len(value)<2:
        return "0"+value
    else:
        return value
    
location_code['Location_Code'] = location_code['Location_Code'].apply(padding)

/Data/Import/doc/location_code_dictionary.csv


## October 2016 Data

### Report 1

In [7]:
in_path_oct_2016_report = in_path
out_path_oct_2016_report = out_path

files = [file for file in db.list_files(in_path_oct_2016_report) if '.xlsx' in file]
files

['CR_AllRecords.xlsx']

In [8]:
saving_files = [file.replace(" ","_").replace(".xlsx","") for file in files]
saving_files

['CR_AllRecords']

IS Investigation Unit Equivalent to Investigator_Assignment?

In [9]:
file=files[0]
final_df = pd.DataFrame()
metadata_df = pd.DataFrame()
#df = pd.read_excel(in_path_oct_2016_report + file)
df = db.download_file(in_path_oct_2016_report,file)
print(df.head())
df.columns = ["Officer_Name","Officer_Unit","Date_of_Appointment","CRID",
                  "Initial_Complaint_Category","Final_Complaint_Category","Incident_Date", 
                  "Complaint_Date","Closed_Date",
               "Final_Finding", "Final_Action_Taken","Final_Action_Description"]

final_df = final_df.append(df)
final_df.reset_index(drop=True,inplace=True)
metadata_df = metadata_df.append(metadata_dataset(df,file))
metadata_df.reset_index(drop=True,inplace=True)

/Data/Import/input/complaints-cpd-2016-oct/cr_allrecords.xlsx
                NAME UNIT DATE_OF_APPOINTMENT   CR_NO  \
0  ABBATE, ANTHONY G  011         05-DEC-1994  237004   
1  ABBATE, ANTHONY G  011         05-DEC-1994  258124   
2   ABBATE, CARMEL G  641         06-JAN-1969  190369   
3   ABBATE, CARMEL G  640         06-JAN-1969  210426   
4   ABBATE, CARMEL G  640         06-JAN-1969  219424   

                     INITIAL_COMPLAINT_CATEGORY  \
0                05A   ARRESTEE - DURING ARREST   
1  04E   PRISONERS PROPERTY - INVENTORY/RECEIPT   
2                           03G   MISCELLANEOUS   
3                    10V   INVENTORY PROCEDURES   
4                           09J   MISCELLANEOUS   

                       FINAL_COMPLAINT_CATEGORY INCIDENT_DATE COMPLAINT_DATE  \
0                05B   ARRESTEE - DURING ARREST   21-MAY-1997    21-MAY-1997   
1  04E   PRISONERS PROPERTY - INVENTORY/RECEIPT   19-NOV-1999    20-NOV-1999   
2                           03G   MISCELLANEOUS 

In [11]:
metadata_df.head()

,Original_Dataset,Column_Name,Non_Null_Count,Unique_Count,Object_Type
0,CR_AllRecords.xlsx,Officer_Name,134683,18907,object
1,CR_AllRecords.xlsx,Officer_Unit,134528,170,object
2,CR_AllRecords.xlsx,Date_of_Appointment,134486,1455,object
3,CR_AllRecords.xlsx,CRID,134683,70449,int64
4,CR_AllRecords.xlsx,Initial_Complaint_Category,134683,128,object


In [19]:
final_df.to_csv(out_path_oct_2016_report+saving_files[0]+".csv",index=False)
final_df.to_excel(out_path_oct_2016_report+saving_files[0]+".xlsx",index=False)

metadata_df.to_csv(out_path_oct_2016_report+saving_files[0]+"_metadata.csv",index=False)